In [1]:
import numpy as np
from lanczos import *
from SVDDecomp import *
from utils import *

In [2]:
precompAll()

runLanczos() {'N': 2, 'k': 2, 'usejit': True, 'orth': True, 'verbose': False} Took 2.2329 seconds
